# Battery Remaining Life Prediction feat. Deep Learning

---
This code only works at google Colab.
(If you can do it on your local app, thats fine, for me it was better option).

You must put 3 battery data on your google drive.

Data we use is at : https://data.matr.io/1/

Data file type : .mat

In this code that I used below, 
first I changed .mat to .pkl File
and then I saved those file on google drive.

참고 논문 : Data-driven prediction of battery cycle life before capacity degradation

First code is converting .mat File to .pkl File

In [ ]:
import h5py
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import pickle
from google.colab import drive
drive.mount('/content/drive')

matFilename = '/content/drive/MyDrive/Colab Notebooks/2017-05-12_batchdata_updated_struct_errorcorrect.mat'

f = h5py.File(matFilename)

batch = f['batch']

num_cells = batch['summary'].shape[0]
bat_dict = {}
for i in range(num_cells):
    cl = f[batch['cycle_life'][i,0]][()]
    policy = f[batch['policy_readable'][i,0]][()].tobytes()[::2].decode()
    summary_IR = np.hstack(f[batch['summary'][i,0]]['IR'][0,:].tolist())
    summary_QC = np.hstack(f[batch['summary'][i,0]]['QCharge'][0,:].tolist())
    summary_QD = np.hstack(f[batch['summary'][i,0]]['QDischarge'][0,:].tolist())
    summary_TA = np.hstack(f[batch['summary'][i,0]]['Tavg'][0,:].tolist())
    summary_TM = np.hstack(f[batch['summary'][i,0]]['Tmin'][0,:].tolist())
    summary_TX = np.hstack(f[batch['summary'][i,0]]['Tmax'][0,:].tolist())
    summary_CT = np.hstack(f[batch['summary'][i,0]]['chargetime'][0,:].tolist())
    summary_CY = np.hstack(f[batch['summary'][i,0]]['cycle'][0,:].tolist())
    summary = {'IR': summary_IR, 'QC': summary_QC, 'QD': summary_QD, 'Tavg':
                summary_TA, 'Tmin': summary_TM, 'Tmax': summary_TX, 'chargetime': summary_CT,
                'cycle': summary_CY}
    cycles = f[batch['cycles'][i,0]]
    cycle_dict = {}
    for j in range(cycles['I'].shape[0]):
        I = np.hstack((f[cycles['I'][j,0]][()]))
        Qc = np.hstack((f[cycles['Qc'][j,0]][()]))
        Qd = np.hstack((f[cycles['Qd'][j,0]][()]))
        Qdlin = np.hstack((f[cycles['Qdlin'][j,0]][()]))
        T = np.hstack((f[cycles['T'][j,0]][()]))
        Tdlin = np.hstack((f[cycles['Tdlin'][j,0]][()]))
        V = np.hstack((f[cycles['V'][j,0]][()]))
        dQdV = np.hstack((f[cycles['discharge_dQdV'][j,0]][()]))
        t = np.hstack((f[cycles['t'][j,0]][()]))
        cd = {'I': I, 'Qc': Qc, 'Qd': Qd, 'Qdlin': Qdlin, 'T': T, 'Tdlin': Tdlin, 'V':V, 'dQdV': dQdV, 't':t}
        cycle_dict[str(j)] = cd

    cell_dict = {'cycle_life': cl, 'charge_policy':policy, 'summary': summary, 'cycles': cycle_dict}
    key = 'b1c' + str(i)
    bat_dict[key]=   cell_dict
with open('batch1.pkl','wb') as fp:
        pickle.dump(bat_dict,fp)

loading .pkl files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pickle
import numpy as np

batch1 = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/batch1.pkl', 'rb')) # 46개, 5개 삭제해서 41개로, 그리고 b1c0, b1c1, b1c2, b1c3, b1c4는 batch2 결과와 연결됨
batch2 = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/batch2.pkl', 'rb')) # 48개, 5개 삭제해서 43개로
batch3 = pickle.load(open('/content/drive/MyDrive/Colab Notebooks/batch3.pkl', 'rb')) # 46개, 6개 삭제해서 40개로

del batch1['b1c8']
del batch1['b1c10']
del batch1['b1c12']
del batch1['b1c13']
del batch1['b1c22']

batch1_keys = ['b1c0', 'b1c1', 'b1c2', 'b1c3', 'b1c4']
batch2_keys = ['b2c7', 'b2c8', 'b2c9', 'b2c15', 'b2c16'] # b1c0, b1c1, b1c2, b1c3, b1c4는 batch2 결과와 연결됨, 논문 서포팅을 참조하자

add_len = [662, 981, 1060, 208, 482]

for i, bk in enumerate(batch1_keys):
    batch1[bk]['cycle_life'] = batch1[bk]['cycle_life'] + add_len[i]
    for j in batch1[bk]['summary'].keys():
        if j == 'cycle':
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j] + len(batch1[bk]['summary'][j])))
        else:
            batch1[bk]['summary'][j] = np.hstack((batch1[bk]['summary'][j], batch2[batch2_keys[i]]['summary'][j]))
    last_cycle = len(batch1[bk]['cycles'].keys())
    for j, jk in enumerate(batch2[batch2_keys[i]]['cycles'].keys()):
        batch1[bk]['cycles'][str(last_cycle + j)] = batch2[batch2_keys[i]]['cycles'][jk]

del batch2['b2c7']
del batch2['b2c8']
del batch2['b2c9']
del batch2['b2c15']
del batch2['b2c16']

del batch3['b3c37']
del batch3['b3c2']
del batch3['b3c23']
del batch3['b3c32']
del batch3['b3c38']
del batch3['b3c39']

numBat1 = len(batch1.keys())
numBat2 = len(batch2.keys())
numBat3 = len(batch3.keys())
numBat = numBat1 + numBat2 + numBat3

bat_dict = {**batch1, **batch2, **batch3}

To check the data is well loaded. Try the code below. 
If it works, then it means you did it! 

In [ ]:
bat_dict['b1c0']['cycle_life']
bat_dict['b1c0']['charge_policy']
bat_dict['b1c0']['summary']
bat_dict['b1c0']['cycles'].keys()

To read Data,
first you need to understand what 'b1c0' means.  

b means battery  

1 means the date the experiement was taken  

c means cell  

0 mean cell number  

each battery have 'cycle_life', 'charge_policy', 'summary', 'cycles' and you can browse it with the method .keys()

In [ ]:
import matplotlib.pyplot as plt

#dict_keys(['I', 'Qc', 'Qd', 'Qdlin', 'T', 'Tdlin', 'V', 'dQdV', 't'])

plt.scatter(bat_dict['b1c1']['cycles']['7']['t'], bat_dict['b1c1']['cycles']['7']['I'], s=1, color='#1A8250')

plt.xlabel('Time (t)', size=12, labelpad=10)
plt.ylabel('Current (I)', size=12, labelpad=10)
plt.yticks(size=12)
plt.xticks(size=12)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

for i in bat_dict['b1c30']['cycles']:
    voltage = bat_dict['b1c30']['cycles'][i]['V']
    capacity = bat_dict['b1c30']['cycles'][i]['Qd']
    plt.scatter(voltage, capacity, s=1)

    # V가 2.0 미만이거나 Qd가 0.1 이하인 값 제거
    filtered_voltage = []
    filtered_capacity = []
    for v, qd in zip(voltage, capacity):
        if v >= 2.0 and qd > 1:  # Qd의 필터링 조건을 0.1 이상으로 변경
            filtered_voltage.append(v)
            filtered_capacity.append(qd)

plt.xlabel('Voltage (V)', size=12, labelpad=10)
plt.ylabel('Dischargecapacity (Qd)', size=12, labelpad=10)
plt.yticks(size=12)
plt.xticks(size=12)

# x축 시작점을 2.0에서 시작하도록 설정
plt.xlim(2.0)  # max(filtered_voltage) + 0.1을 사용하여 x축의 끝 범위를 조금 더 넓혀주어 모든 데이터 포인트를 확실히 보여줌
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(9, 3))

for i in bat_dict.keys():
  plt.scatter(bat_dict[i]['summary']['cycle'], bat_dict[i]['summary']['QD'], s=3)

plt.xlim(0, 1000)
plt.ylim(0.88, 1.10)
plt.show()

Below is the battery prediction code 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from keras import models
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import stats

x_train, x_test, y_train, y_test = train_test_split(q10_100_ml, cycles_group, test_size=0.2, random_state=2)

x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

transformer = StandardScaler()
transformer.fit(x_train)

x_train = transformer.transform(x_train)
x_test = transformer.transform(x_test)

model = models.Sequential()
model.add(layers.Dense(50, input_dim=70, activation='relu'))
model.add(layers.Dense(30, activation='relu'))
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mae', metrics='mse')
#model.fit(x_train, y_train, batch_size=9, epochs=80)

history = model.fit(x_train, y_train, batch_size=9, epochs=200, validation_data=(x_test, y_test))

plt.figure(figsize=(14, 10))
plt.plot(history.history['loss'], color='g')
plt.plot(history.history['val_loss'], color='r')
plt.xlabel('Epoch', size=22, labelpad=10)
plt.ylabel('Loss', size=22, labelpad=10)

plt.yticks(size=16)
plt.xticks(size=16)
plt.legend(['train','test'], frameon=False, fontsize=22)
plt.show()

plt.figure(figsize=(14, 10))

expt_val = []
pred_val = []

for i in range(len(y_test)):
    y_pred = model.predict(x_test)
#    print("정답값 = {}, 예측값 = {}".format(y_test[i], y_pred[i][0]))
    plt.scatter(y_test[i], y_pred[i][0], color='g')
    expt_val.append(y_test[i])
    pred_val.append(y_pred[i][0])

plt.plot([0, 3000], [0, 3000], 'c--')
plt.xlim(0, 2500)
plt.ylim(0, 2500)
plt.xlabel('Experimental Cycle Life', size=22, labelpad=10)
plt.ylabel('Predicted Cycle Life', size=22, labelpad=10)
plt.yticks(size=16)
plt.xticks(size=16)
plt.show()

pc = stats.pearsonr(expt_val, pred_val)
print(pc)